In [1]:
import sys
sys.path.insert(1, '..')
import importlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sqlalchemy import create_engine

from snorkel.labeling import labeling_function

## Load Data

### DataFrame

In [2]:
## load the data by choosing a random 10% of the data
## is the data ordered? am I missing a large chunk due to my partial loading?
# p = 0.01
# responses = pd.read_csv('../data/moralmachine/SharedResponses.csv', skiprows=lambda i: i>0 and random.random() > p)

### SQL Engine

In [3]:
# access SQL DB with data
engine = create_engine("sqlite:///../data/moralmachine.db", echo=False)

In [4]:
# demo
# responses = pd.read_sql("SELECT * FROM sharedresponses ORDER BY RANDOM() LIMIT 100000", con=engine)
# responses.columns

In [5]:
# ## For viewing the possible values for each feature
# for feature in responses.columns:
#     print(feature)
#     print(responses[feature].unique())
#     print(responses[feature].unique().shape)

In [6]:
# ## What does a single user's session/response look like?
# responses_random = responses[responses['ScenarioTypeStrict'] == 'Random']
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     ## see a random user's session
# #     display(responses_random[responses_random['ExtendedSessionID'] == responses_random['ExtendedSessionID'].sample().values[0]])
    
#     ## see the user with nth most responses
#     n = 100
#     display(responses_random[responses_random['ResponseID'] == responses_random.groupby(by='ResponseID').size().sort_values(ascending=False).index[n]]) 

In [7]:
# ## group alternatives pairwise!
# # take only responses where both alternatives are present in the random sample - for a full sample, this step isn't necessary
# responses_grouped = responses.groupby(by='ResponseID').filter(lambda g: g.shape[0] < 2)
# responses_grouped.shape

## Data Exploration

In [8]:
# responses.describe()

## Snorkel Labeling
Using https://www.snorkel.org/use-cases/01-spam-tutorial

### Sample Data

In [9]:
# query random sample of responses grouped by ResponseID; only take responses for which both instances are present
sample_size = 100000
query = """
    SELECT gc.session_count, sr.* FROM sharedresponses sr
    INNER JOIN (
        SELECT ExtendedSessionID, COUNT(DISTINCT ResponseID) AS session_count FROM sharedresponses
        WHERE UserID <> ''
        
        /* Get only full sessions. */
        GROUP BY ExtendedSessionID
            HAVING COUNT(DISTINCT ResponseID) LIKE 13
                AND COUNT(ResponseID) LIKE 26
        
        LIMIT {0:d}
    ) gc
        ON gc.ExtendedSessionID = sr.ExtendedSessionID
    ORDER BY sr.UserID
""".format(sample_size)
df = pd.read_sql(query, con=engine)
df

session_count         ResponseID              ExtendedSessionID  \
0                   13  C4yoxBdFMTE3tAjEu  1295349308_1000284909295010.0   
1                   13  CDnbR339euts3x3c3  1295349308_1000284909295010.0   
2                   13  CMChvaNp5QZup4QBw  1295349308_1000284909295010.0   
3                   13  CZ3BfpjubdHMdqq6z  1295349308_1000284909295010.0   
4                   13  C4yoxBdFMTE3tAjEu  1295349308_1000284909295010.0   
...                ...                ...                            ...   
1544915             13  oNwjkhRzJpiBHfJNB  1708630306_9999973699194224.0   
1544916             13  zjbT9ydvthWaApyz3  1708630306_9999973699194224.0   
1544917             13  nhxSBhkYqDjNqqeYu  1708630306_9999973699194224.0   
1544918             13  oNwjkhRzJpiBHfJNB  1708630306_9999973699194224.0   
1544919             13  zjbT9ydvthWaApyz3  1708630306_9999973699194224.0   

                   UserID  ScenarioOrder  Intervention  PedPed  Barrier  \
0        1000284909295010              9             0       0        1   
1        1000284909295010              6             0       1        0   
2        1000284909295010              2             0       0        1   
3        1000284909295010              5             0       1        0   
4        1000284909295010              9             1       0        0   
...                   ...            ...           ...     ...      ...   
1544915  9999973699194220              4             0       1        0   
1544916  9999973699194220             12             0       0        1   
1544917  9999973699194220              3             1       1        0   
1544918  9999973699194220              4             1       1        0   
1544919  9999973699194220             12             1       0        0   

         CrossingSignal AttributeLevel  ... LargeMan Criminal MaleExecutive  \
0                     0            Fat  ...        0        0             0   
1                     0           Less  ...        0        0             0   
2                     0           Rand  ...        1        0             0   
3                     0          Young  ...        0        0             0   
4                     0            Fit  ...        0        0             0   
...                 ...            ...  ...      ...      ...           ...   
1544915               0            Fit  ...        0        0             0   
1544916               0            Old  ...        0        0             0   
1544917               0           High  ...        0        0             0   
1544918               0            Fat  ...        0        0             0   
1544919               0          Young  ...        0        0             0   

        FemaleExecutive FemaleAthlete  MaleAthlete  FemaleDoctor  MaleDoctor  \
0                     0             0            0             0           0   
1                     0             0            0             1           0   
2                     1             0            0             0           0   
3                     0             0            0             0           0   
4                     0             1            2             0           0   
...                 ...           ...          ...           ...         ...   
1544915               0             1            1             0           0   
1544916               0             0            0             0           0   
1544917               0             0            0             0           0   
1544918               0             0            0             0           0   
1544919               0             0            0             0           0   

        Dog Cat  
0         0   0  
1         0   0  
2         0   0  
3         0   0  
4         0   0  
...      ..  ..  
1544915   0   0  
1544916   0   0  
1544917   0   0  
1544918   0   0  
1544919   0   0  

[1544920 rows x 42 columns]

In [10]:
# size and other stuff
df.describe()

session_count  ScenarioOrder  Intervention        PedPed       Barrier  \
count      1544920.0   1.544920e+06     1544920.0  1.544920e+06  1.544920e+06   
mean            13.0   6.995973e+00           0.5  4.525270e-01  2.737365e-01   
std              0.0   3.742443e+00           0.5  4.977414e-01  4.458755e-01   
min             13.0   1.000000e+00           0.0  0.000000e+00  0.000000e+00   
25%             13.0   4.000000e+00           0.0  0.000000e+00  0.000000e+00   
50%             13.0   7.000000e+00           0.5  0.000000e+00  0.000000e+00   
75%             13.0   1.000000e+01           1.0  1.000000e+00  1.000000e+00   
max             13.0   1.300000e+01           1.0  1.000000e+00  1.000000e+00   

       CrossingSignal  NumberOfCharacters  DiffNumberOFCharacters      Saved  \
count    1.544920e+06        1.544920e+06            1.544920e+06  1544920.0   
mean     6.062489e-01        2.901300e+00            5.101623e-01        0.5   
std      8.149253e-01        1.492986e+00            1.099643e+00        0.5   
min      0.000000e+00        1.000000e+00            0.000000e+00        0.0   
25%      0.000000e+00        1.000000e+00            0.000000e+00        0.0   
50%      0.000000e+00        3.000000e+00            0.000000e+00        0.5   
75%      1.000000e+00        4.000000e+00            0.000000e+00        1.0   
max      2.000000e+00        5.000000e+00            4.000000e+00        1.0   

                Man  ...      LargeMan      Criminal  MaleExecutive  \
count  1.544920e+06  ...  1.544920e+06  1.544920e+06   1.544920e+06   
mean   3.132848e-01  ...  1.441291e-01  5.329079e-02   1.073615e-01   
std    5.904199e-01  ...  4.278764e-01  2.423233e-01   3.480164e-01   
min    0.000000e+00  ...  0.000000e+00  0.000000e+00   0.000000e+00   
25%    0.000000e+00  ...  0.000000e+00  0.000000e+00   0.000000e+00   
50%    0.000000e+00  ...  0.000000e+00  0.000000e+00   0.000000e+00   
75%    1.000000e+00  ...  0.000000e+00  0.000000e+00   0.000000e+00   
max    5.000000e+00  ...  5.000000e+00  4.000000e+00   5.000000e+00   

       FemaleExecutive  FemaleAthlete   MaleAthlete  FemaleDoctor  \
count     1.544920e+06   1.544920e+06  1.544920e+06  1.544920e+06   
mean      1.071712e-01   1.731565e-01  1.724141e-01  9.103190e-02   
std       3.474571e-01   5.008922e-01  4.997909e-01  3.227842e-01   
min       0.000000e+00   0.000000e+00  0.000000e+00  0.000000e+00   
25%       0.000000e+00   0.000000e+00  0.000000e+00  0.000000e+00   
50%       0.000000e+00   0.000000e+00  0.000000e+00  0.000000e+00   
75%       0.000000e+00   0.000000e+00  0.000000e+00  0.000000e+00   
max       5.000000e+00   5.000000e+00  5.000000e+00  5.000000e+00   

         MaleDoctor           Dog           Cat  
count  1.544920e+06  1.544920e+06  1.544920e+06  
mean   9.081765e-02  1.561356e-01  1.564081e-01  
std    3.221732e-01  5.405188e-01  5.417887e-01  
min    0.000000e+00  0.000000e+00  0.000000e+00  
25%    0.000000e+00  0.000000e+00  0.000000e+00  
50%    0.000000e+00  0.000000e+00  0.000000e+00  
75%    0.000000e+00  0.000000e+00  0.000000e+00  
max    5.000000e+00  5.000000e+00  5.000000e+00  

[8 rows x 29 columns]

In [11]:
# number of users
df['UserID'].nunique()

51211

For comparison to Noothigattu et al., how many pairwise comparisons per voter in this sample dataset?

In [12]:
# number of pairwise comparisons per voter?
# = number of response IDs per voter
df.groupby('UserID')['ResponseID'].nunique().mean()

15.083868700083967

In [13]:
# number of scenarios
pd.DataFrame(df['ScenarioType'].value_counts()/df['ScenarioType'].value_counts().sum()).to_csv("../figures/data/freq_scenario.csv")
# frequency of each character count
characters = ['Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat']
pd.DataFrame(df[characters].sum()/df[characters].sum().sum(), columns=['frequency']).to_csv('../figures/data/freq_character.csv')

In [14]:
# user countries
freqs = df['UserCountry3'].value_counts()
pd.DataFrame(freqs/freqs.sum()).to_csv('../figures/data/freq_countries.csv')

In [15]:
df.iloc[:26,:].sort_values('ScenarioOrder')

session_count         ResponseID              ExtendedSessionID  \
12             13  P3nEs8eAZDgegGQgL  1295349308_1000284909295010.0   
9              13  P3nEs8eAZDgegGQgL  1295349308_1000284909295010.0   
2              13  CMChvaNp5QZup4QBw  1295349308_1000284909295010.0   
6              13  CMChvaNp5QZup4QBw  1295349308_1000284909295010.0   
17             13  mYhYL5Gxwpg53whqt  1295349308_1000284909295010.0   
15             13  mYhYL5Gxwpg53whqt  1295349308_1000284909295010.0   
22             13  rqrbkEMFStk5znStw  1295349308_1000284909295010.0   
18             13  rqrbkEMFStk5znStw  1295349308_1000284909295010.0   
3              13  CZ3BfpjubdHMdqq6z  1295349308_1000284909295010.0   
7              13  CZ3BfpjubdHMdqq6z  1295349308_1000284909295010.0   
1              13  CDnbR339euts3x3c3  1295349308_1000284909295010.0   
5              13  CDnbR339euts3x3c3  1295349308_1000284909295010.0   
20             13  yfZ8upfmyuDxkvPCF  1295349308_1000284909295010.0   
24             13  yfZ8upfmyuDxkvPCF  1295349308_1000284909295010.0   
8              13  Kvr4yzcYB8QAxykMg  1295349308_1000284909295010.0   
11             13  Kvr4yzcYB8QAxykMg  1295349308_1000284909295010.0   
0              13  C4yoxBdFMTE3tAjEu  1295349308_1000284909295010.0   
4              13  C4yoxBdFMTE3tAjEu  1295349308_1000284909295010.0   
14             13  dds6m4CyrbAYYZhXp  1295349308_1000284909295010.0   
16             13  dds6m4CyrbAYYZhXp  1295349308_1000284909295010.0   
19             13  vhFv3vcHHNzQJzKvt  1295349308_1000284909295010.0   
23             13  vhFv3vcHHNzQJzKvt  1295349308_1000284909295010.0   
13             13  XFna5WHLT9rXd37rE  1295349308_1000284909295010.0   
10             13  XFna5WHLT9rXd37rE  1295349308_1000284909295010.0   
21             13  z8BcEcp7cHkxFiTGo  1295349308_1000284909295010.0   
25             13  z8BcEcp7cHkxFiTGo  1295349308_1000284909295010.0   

              UserID  ScenarioOrder  Intervention  PedPed  Barrier  \
12  1000284909295010              1             1       1        0   
9   1000284909295010              1             0       1        0   
2   1000284909295010              2             0       0        1   
6   1000284909295010              2             1       0        0   
17  1000284909295010              3             1       0        0   
15  1000284909295010              3             0       0        1   
22  1000284909295010              4             1       0        1   
18  1000284909295010              4             0       0        0   
3   1000284909295010              5             0       1        0   
7   1000284909295010              5             1       1        0   
1   1000284909295010              6             0       1        0   
5   1000284909295010              6             1       1        0   
20  1000284909295010              7             0       1        0   
24  1000284909295010              7             1       1        0   
8   1000284909295010              8             0       1        0   
11  1000284909295010              8             1       1        0   
0   1000284909295010              9             0       0        1   
4   1000284909295010              9             1       0        0   
14  1000284909295010             10             0       0        0   
16  1000284909295010             10             1       0        1   
19  1000284909295010             11             0       0        0   
23  1000284909295010             11             1       0        1   
13  1000284909295010             12             1       1        0   
10  1000284909295010             12             0       1        0   
21  1000284909295010             13             0       1        0   
25  1000284909295010             13             1       1        0   

    CrossingSignal AttributeLevel  ... LargeMan Criminal MaleExecutive  \
12               1            Low  ...        0        0             0   
9                2           High  ...        0

Out of the variables above, these are the variables that vary within response pairs:
> 'NumberOfCharacters', 'DiffNumberOfCharacters', 'Saved', 'Template', 'DescriptionShown',
'LeftHand', 'UserCountry3', 'Man', 'Woman', 'Pregnant', 'Stroller',
'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman',
'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive',
'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog',
'Cat'

N.B. in each pair of responses, only one is the result of an intervention, and only one is saved. 

In [16]:
df[["ResponseID","Intervention","Saved"]].head()

ResponseID  Intervention  Saved
0  C4yoxBdFMTE3tAjEu             0      1
1  CDnbR339euts3x3c3             0      0
2  CMChvaNp5QZup4QBw             0      1
3  CZ3BfpjubdHMdqq6z             0      1
4  C4yoxBdFMTE3tAjEu             1      0

### Preprocessing

First, let's convert to abstract features for ease of labeling.

In [17]:
import hmm.labeling.utils
importlib.reload(hmm.labeling.utils)
from hmm.labeling.utils import transform_abstract

ids = ['ResponseID', 'ExtendedSessionID', 'UserID']
df_i = df.set_index(ids, append=True, verify_integrity=True)
df_i.index = df_i.index.set_names(['UUID']+ids)
df_abstract = transform_abstract(df_i)
df_abstract.head()

session_count  \
UUID ResponseID        ExtendedSessionID             UserID                            
0    C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010             13   
1    CDnbR339euts3x3c3 1295349308_1000284909295010.0 1000284909295010             13   
2    CMChvaNp5QZup4QBw 1295349308_1000284909295010.0 1000284909295010             13   
3    CZ3BfpjubdHMdqq6z 1295349308_1000284909295010.0 1000284909295010             13   
4    C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010             13   

                                                                       ScenarioOrder  \
UUID ResponseID        ExtendedSessionID             UserID                            
0    C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010              9   
1    CDnbR339euts3x3c3 1295349308_1000284909295010.0 1000284909295010              6   
2    CMChvaNp5QZup4QBw 1295349308_1000284909295010.0 1000284909295010              2   
3    CZ3BfpjubdHMdqq6z 1295349308_1000284909295010.0 1000284909295010              5   
4    C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010              9   

                                                                       PedPed  \
UUID ResponseID        ExtendedSessionID             UserID                     
0    C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010       0   
1    CDnbR339euts3x3c3 1295349308_1000284909295010.0 1000284909295010       1   
2    CMChvaNp5QZup4QBw 1295349308_1000284909295010.0 1000284909295010       0   
3    CZ3BfpjubdHMdqq6z 1295349308_1000284909295010.0 1000284909295010       1   
4    C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010       0   

                                                                       Barrier  \
UUID ResponseID        ExtendedSessionID             UserID                      
0    C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010        1   
1    CDnbR339euts3x3c3 1295349308_1000284909295010.0 1000284909295010        0   
2    CMChvaNp5QZup4QBw 1295349308_1000284909295010.0 1000284909295010        1   
3    CZ3BfpjubdHMdqq6z 1295349308_1000284909295010.0 1000284909295010        0   
4    C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010        0   

                                                                       CrossingSignal  \
UUID ResponseID        ExtendedSessionID             UserID                             
0    C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010               0   
1    CDnbR339euts3x3c3 1295349308_1000284909295010.0 1000284909295010               0   
2    CMChvaNp5QZup4QBw 1295349308_1000284909295010.0 1000284909295010               0   
3    CZ3BfpjubdHMdqq6z 1295349308_1000284909295010.0 1000284909295010               0   
4    C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010               0   

                                                                      AttributeLevel  \
UUID ResponseID        ExtendedSessionID             UserID                            
0    C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010            Fat   
1    CDnbR339euts3x3c3 1295349308_1000284909295010.0 1000284909295010           Less   
2    CMChvaNp5QZup4QBw 1295349308_1000284909295010.0 1000284909295010           Rand   
3    CZ3BfpjubdHMdqq6z 1295349308_1000284909295010.0 1000284909295010          Young   
4    C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010            Fit   

                                                                      ScenarioTypeStrict  \
UUID ResponseID        ExtendedSessionID             UserID                                
0    C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010            Fitness   
1    CDnbR339euts3x3c3 1295349308_1000284909295010.0 1000284909295010        Utilitarian   
2    CMChvaNp5QZup4QBw 1295349308_1000284909295010.0 1000284909295010             Ran

Now let's check our work.

In [18]:
sample = df_i.sample().index
df_i.loc[sample]

session_count  \
UUID   ResponseID        ExtendedSessionID        UserID                      
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726             13   

                                                              ScenarioOrder  \
UUID   ResponseID        ExtendedSessionID        UserID                      
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726             13   

                                                              Intervention  \
UUID   ResponseID        ExtendedSessionID        UserID                     
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726             1   

                                                              PedPed  Barrier  \
UUID   ResponseID        ExtendedSessionID        UserID                        
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726       1        0   

                                                              CrossingSignal  \
UUID   ResponseID        ExtendedSessionID        UserID                       
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726               0   

                                                             AttributeLevel  \
UUID   ResponseID        ExtendedSessionID        UserID                      
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726            Low   

                                                             ScenarioTypeStrict  \
UUID   ResponseID        ExtendedSessionID        UserID                          
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726      Social Status   

                                                               ScenarioType  \
UUID   ResponseID        ExtendedSessionID        UserID                      
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726  Social Status   

                                                             DefaultChoice  \
UUID   ResponseID        ExtendedSessionID        UserID                     
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726          High   

                                                              ...  \
UUID   ResponseID        ExtendedSessionID        UserID      ...   
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726  ...   

                                                             FemaleExecutive  \
UUID   ResponseID        ExtendedSessionID        UserID                       
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726               0   

                                                             FemaleAthlete  \
UUID   ResponseID        ExtendedSessionID        UserID                     
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726             0   

                                                              MaleAthlete  \
UUID   ResponseID        ExtendedSessionID        UserID                    
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726            0   

                                                              FemaleDoctor  \
UUID   ResponseID        ExtendedSessionID        UserID                     
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726             0   

                                                              MaleDoctor Dog  \
UUID   ResponseID        ExtendedSessionID        UserID                       
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726           0   0   

                                                             Cat Passenger  \
UUID   ResponseID        ExtendedSessionID        UserID                     
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726   0     False   

                                                             Law Abiding  \
UUID   ResponseID        ExtendedSessionID        UserID                   
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726       False   

                                                 

In [19]:
df_abstract.loc[sample]

session_count  \
UUID   ResponseID        ExtendedSessionID        UserID                      
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726             13   

                                                              ScenarioOrder  \
UUID   ResponseID        ExtendedSessionID        UserID                      
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726             13   

                                                              PedPed  Barrier  \
UUID   ResponseID        ExtendedSessionID        UserID                        
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726       1        0   

                                                              CrossingSignal  \
UUID   ResponseID        ExtendedSessionID        UserID                       
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726               0   

                                                             AttributeLevel  \
UUID   ResponseID        ExtendedSessionID        UserID                      
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726            Low   

                                                             ScenarioTypeStrict  \
UUID   ResponseID        ExtendedSessionID        UserID                          
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726      Social Status   

                                                               ScenarioType  \
UUID   ResponseID        ExtendedSessionID        UserID                      
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726  Social Status   

                                                             DefaultChoice  \
UUID   ResponseID        ExtendedSessionID        UserID                     
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726          High   

                                                             NonDefaultChoice  \
UUID   ResponseID        ExtendedSessionID        UserID                        
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726              Low   

                                                              ... Fit  \
UUID   ResponseID        ExtendedSessionID        UserID      ...       
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726  ...   0   

                                                              Working  \
UUID   ResponseID        ExtendedSessionID        UserID                
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726        0   

                                                              Medical  \
UUID   ResponseID        ExtendedSessionID        UserID                
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726        0   

                                                              Homeless  \
UUID   ResponseID        ExtendedSessionID        UserID                 
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726         1   

                                                             Criminal Human  \
UUID   ResponseID        ExtendedSessionID        UserID                      
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726        0     1   

                                                             Non-human  \
UUID   ResponseID        ExtendedSessionID        UserID                 
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726         0   

                                                             Passenger  \
UUID   ResponseID        ExtendedSessionID        UserID                 
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726         0   

                                                             Law Abiding  \
UUID   ResponseID        ExtendedSessionID        UserID                   
850425 ZqCfW7GJsekhxYhWr -1863744697_4210485726.0 4210485726           0   

                                                             Law Violating  
UUID   ResponseID      

In [20]:
df_abstract.columns

Index(['session_count', 'ScenarioOrder', 'PedPed', 'Barrier', 'CrossingSignal',
       'AttributeLevel', 'ScenarioTypeStrict', 'ScenarioType', 'DefaultChoice',
       'NonDefaultChoice', 'DefaultChoiceIsOmission', 'NumberOfCharacters',
       'DiffNumberOFCharacters', 'Saved', 'Template', 'DescriptionShown',
       'LeftHand', 'UserCountry3', 'Intervene', 'Male', 'Female', 'Young',
       'Old', 'Infancy', 'Pregnancy', 'Fat', 'Fit', 'Working', 'Medical',
       'Homeless', 'Criminal', 'Human', 'Non-human', 'Passenger',
       'Law Abiding', 'Law Violating'],
      dtype='object')

First, select the fields that are unique to each scenario (the fields that vary within pairs of responses). Then split the dataset into two disjoint sets of alternatives: one in which an intervention occurs, and one in which there is no intervention.

In [21]:
ids = ["ResponseID", "ExtendedSessionID", "UserID"]
scenario_fields = [
    'Saved', 'NumberOfCharacters', 'DescriptionShown', 'LeftHand', 'Male', 'Female', 'Young', 'Old', 'Infancy', 'Pregnancy',
    'Fat', 'Fit', 'Working', 'Medical', 'Homeless', 'Criminal', 'Human',
    'Non-human', 'Passenger', 'Law Abiding', 'Law Violating'
]

df_abstract = df_abstract.reset_index(level='UUID')
sample_response = df_abstract.sample().index
intervention = df_abstract[df_abstract['Intervene'] == 1][scenario_fields]
no_intervention = df_abstract[df_abstract['Intervene'] == 0][scenario_fields]

print("Alternative w/ intervention:")
display(intervention.loc[sample_response])
print("Alternative w/o intervention:")
display(no_intervention.loc[sample_response])

Alternative w/ intervention:


Saved  \
ResponseID        ExtendedSessionID             UserID                    
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250      1   

                                                                  NumberOfCharacters  \
ResponseID        ExtendedSessionID             UserID                                 
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250                   2   

                                                                 DescriptionShown  \
ResponseID        ExtendedSessionID             UserID                              
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250                0   

                                                                 LeftHand  \
ResponseID        ExtendedSessionID             UserID                      
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250        1   

                                                                 Male Female  \
ResponseID        ExtendedSessionID             UserID                         
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250    0      0   

                                                                 Young Old  \
ResponseID        ExtendedSessionID             UserID                       
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250     0   0   

                                                                 Infancy  \
ResponseID        ExtendedSessionID             UserID                     
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250       0   

                                                                 Pregnancy  \
ResponseID        ExtendedSessionID             UserID                       
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250         0   

                                                                  ... Fit  \
ResponseID        ExtendedSessionID             UserID            ...       
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250  ...   0   

                                                                 Working  \
ResponseID        ExtendedSessionID             UserID                     
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250       0   

                                                                 Medical  \
ResponseID        ExtendedSessionID             UserID                     
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250       0   

                                                                 Homeless  \
ResponseID        ExtendedSessionID             UserID                      
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250        0   

                                                                 Criminal  \
ResponseID        ExtendedSessionID             UserID                      
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250        0   

                                                                 Human  \
ResponseID        ExtendedSessionID             UserID                   
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250     0   

                                                                 Non-human  \
ResponseID        ExtendedSessionID             UserID                       
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250         2   

                                                                 Passenger  \
ResponseID        ExtendedSessionID             UserID                       
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250         1   

                                                                 Law Abiding  \
ResponseID        ExtendedSessionID             UserID                         
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250           1   

                                                           

Alternative w/o intervention:


Saved  \
ResponseID        ExtendedSessionID             UserID                    
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250      0   

                                                                  NumberOfCharacters  \
ResponseID        ExtendedSessionID             UserID                                 
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250                   2   

                                                                 DescriptionShown  \
ResponseID        ExtendedSessionID             UserID                              
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250                0   

                                                                 LeftHand  \
ResponseID        ExtendedSessionID             UserID                      
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250        0   

                                                                 Male Female  \
ResponseID        ExtendedSessionID             UserID                         
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250    1      1   

                                                                 Young Old  \
ResponseID        ExtendedSessionID             UserID                       
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250     0   2   

                                                                 Infancy  \
ResponseID        ExtendedSessionID             UserID                     
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250       0   

                                                                 Pregnancy  \
ResponseID        ExtendedSessionID             UserID                       
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250         0   

                                                                  ... Fit  \
ResponseID        ExtendedSessionID             UserID            ...       
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250  ...   0   

                                                                 Working  \
ResponseID        ExtendedSessionID             UserID                     
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250       0   

                                                                 Medical  \
ResponseID        ExtendedSessionID             UserID                     
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250       0   

                                                                 Homeless  \
ResponseID        ExtendedSessionID             UserID                      
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250        0   

                                                                 Criminal  \
ResponseID        ExtendedSessionID             UserID                      
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250        0   

                                                                 Human  \
ResponseID        ExtendedSessionID             UserID                   
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250     2   

                                                                 Non-human  \
ResponseID        ExtendedSessionID             UserID                       
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250         0   

                                                                 Passenger  \
ResponseID        ExtendedSessionID             UserID                       
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250         0   

                                                                 Law Abiding  \
ResponseID        ExtendedSessionID             UserID                         
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250           0   

                                                           

Next, combine the datasets on response ID, separating the variable characteristics with suffixes.

In [22]:
df_endo = intervention.join(no_intervention, lsuffix='_int', rsuffix='_noint', how='inner')
df_endo.loc[sample_response]

Saved_int  \
ResponseID        ExtendedSessionID             UserID                        
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250          1   

                                                                  NumberOfCharacters_int  \
ResponseID        ExtendedSessionID             UserID                                     
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250                       2   

                                                                 DescriptionShown_int  \
ResponseID        ExtendedSessionID             UserID                                  
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250                    0   

                                                                 LeftHand_int  \
ResponseID        ExtendedSessionID             UserID                          
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250            1   

                                                                 Male_int  \
ResponseID        ExtendedSessionID             UserID                      
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250        0   

                                                                 Female_int  \
ResponseID        ExtendedSessionID             UserID                        
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250          0   

                                                                 Young_int  \
ResponseID        ExtendedSessionID             UserID                       
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250         0   

                                                                 Old_int  \
ResponseID        ExtendedSessionID             UserID                     
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250       0   

                                                                 Infancy_int  \
ResponseID        ExtendedSessionID             UserID                         
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250           0   

                                                                 Pregnancy_int  \
ResponseID        ExtendedSessionID             UserID                           
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250             0   

                                                                  ...  \
ResponseID        ExtendedSessionID             UserID            ...   
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250  ...   

                                                                 Fit_noint  \
ResponseID        ExtendedSessionID             UserID                       
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250         0   

                                                                 Working_noint  \
ResponseID        ExtendedSessionID             UserID                           
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250             0   

                                                                 Medical_noint  \
ResponseID        ExtendedSessionID             UserID                           
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250             0   

                                                                 Homeless_noint  \
ResponseID        ExtendedSessionID             UserID                            
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250              0   

                                                                 Criminal_noint  \
ResponseID        ExtendedSessionID             UserID                            
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250              0   

                                                                 Human_noint  \
ResponseID        ExtendedSessionID             UserID                         
ncZwLgeo

Get data for the columns from the raw data that _didn't_ change within response pairs. Remember to eliminate duplicate response pairs - now that we have a combined tuple for each pairwise comparison, they're no longer necessary.

In [23]:
df_exo = df_abstract[[col for col in df_abstract.columns if col not in scenario_fields]]
df_exo = df_exo.loc[~df_exo.index.duplicated(keep='first')]
df_exo.loc[sample_response]

UUID  \
ResponseID        ExtendedSessionID             UserID                      
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250  1318350   

                                                                  session_count  \
ResponseID        ExtendedSessionID             UserID                            
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250             13   

                                                                  ScenarioOrder  \
ResponseID        ExtendedSessionID             UserID                            
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250             12   

                                                                  PedPed  \
ResponseID        ExtendedSessionID             UserID                     
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250       0   

                                                                  Barrier  \
ResponseID        ExtendedSessionID             UserID                      
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250        0   

                                                                  CrossingSignal  \
ResponseID        ExtendedSessionID             UserID                             
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250               2   

                                                                 AttributeLevel  \
ResponseID        ExtendedSessionID             UserID                            
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250        Hoomans   

                                                                 ScenarioTypeStrict  \
ResponseID        ExtendedSessionID             UserID                                
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250            Species   

                                                                 ScenarioType  \
ResponseID        ExtendedSessionID             UserID                          
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250      Species   

                                                                 DefaultChoice  \
ResponseID        ExtendedSessionID             UserID                           
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250       Hoomans   

                                                                 NonDefaultChoice  \
ResponseID        ExtendedSessionID             UserID                              
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250             Pets   

                                                                 DefaultChoiceIsOmission  \
ResponseID        ExtendedSessionID             UserID                                     
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250                       1   

                                                                  DiffNumberOFCharacters  \
ResponseID        ExtendedSessionID             UserID                                     
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250                       0   

                                                                 Template  \
ResponseID        ExtendedSessionID             UserID                      
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250  Desktop   

                                                                 UserCountry3  \
ResponseID        ExtendedSessionID             UserID                          
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250          GRC   

                                                                 Intervene  
ResponseID        ExtendedSessionID             UserID                      
ncZwLgeoMbQNN4tiy 1025948944_8037765365149250.0 8037765365149250         0

Then join that data in with the combined endogenous variables to get a full tuple for each pairwise comparison presented to a user.

In [24]:
df_joined = df_exo.join(df_endo, how='inner').set_index(['UUID'], append=True, verify_integrity=True)
sample = df_joined.sample().index
df_joined.loc[sample]

session_count  \
ResponseID        ExtendedSessionID            UserID           UUID                    
PnQHToK52EJ6nQ4wb 471131761_3683297797364844.0 3683297797364840 713863             13   

                                                                        ScenarioOrder  \
ResponseID        ExtendedSessionID            UserID           UUID                    
PnQHToK52EJ6nQ4wb 471131761_3683297797364844.0 3683297797364840 713863              3   

                                                                        PedPed  \
ResponseID        ExtendedSessionID            UserID           UUID             
PnQHToK52EJ6nQ4wb 471131761_3683297797364844.0 3683297797364840 713863       1   

                                                                        Barrier  \
ResponseID        ExtendedSessionID            UserID           UUID              
PnQHToK52EJ6nQ4wb 471131761_3683297797364844.0 3683297797364840 713863        0   

                                                                        CrossingSignal  \
ResponseID        ExtendedSessionID            UserID           UUID                     
PnQHToK52EJ6nQ4wb 471131761_3683297797364844.0 3683297797364840 713863               0   

                                                                       AttributeLevel  \
ResponseID        ExtendedSessionID            UserID           UUID                    
PnQHToK52EJ6nQ4wb 471131761_3683297797364844.0 3683297797364840 713863           Less   

                                                                       ScenarioTypeStrict  \
ResponseID        ExtendedSessionID            UserID           UUID                        
PnQHToK52EJ6nQ4wb 471131761_3683297797364844.0 3683297797364840 713863        Utilitarian   

                                                                       ScenarioType  \
ResponseID        ExtendedSessionID            UserID           UUID                  
PnQHToK52EJ6nQ4wb 471131761_3683297797364844.0 3683297797364840 713863  Utilitarian   

                                                                       DefaultChoice  \
ResponseID        ExtendedSessionID            UserID           UUID                   
PnQHToK52EJ6nQ4wb 471131761_3683297797364844.0 3683297797364840 713863          More   

                                                                       NonDefaultChoice  \
ResponseID        ExtendedSessionID            UserID           UUID                      
PnQHToK52EJ6nQ4wb 471131761_3683297797364844.0 3683297797364840 713863             Less   

                                                                        ...  \
ResponseID        ExtendedSessionID            UserID           UUID    ...   
PnQHToK52EJ6nQ4wb 471131761_3683297797364844.0 3683297797364840 713863  ...   

                                                                       Fit_noint  \
ResponseID        ExtendedSessionID            UserID           UUID               
PnQHToK52EJ6nQ4wb 471131761_3683297797364844.0 3683297797364840 713863         0   

                                                                        Working_noint  \
ResponseID        ExtendedSessionID            UserID           UUID                    
PnQHToK52EJ6nQ4wb 471131761_3683297797364844.0 3683297797364840 713863              0   

                                                                       Medical_noint  \
ResponseID        ExtendedSessionID            UserID           UUID                   
PnQHToK52EJ6nQ4wb 471131761_3683297797364844.0 3683297797364840 713863             0   

                                                                       Homeless_noint  \
ResponseID        ExtendedSessionID            UserID           UUID                    
PnQHToK52EJ6nQ4wb 471131761_3683297797364844.0 3683297797364840 713863              1   

                                                                       Criminal_noint  \
ResponseID        ExtendedSessionID   

In [25]:
import hmm.labeling.utils
importlib.reload(hmm.labeling.utils)
from hmm.labeling.utils import pictofy
    
# works with abstract
pictofy(df_joined.loc[sample])

What should the self-driving car do? (ScenarioType: Utilitarian)

	    ðŸš˜ 
	   |	\ 
	   v 	 v
	ðŸ”´ðŸš¸ 	ðŸš¸ðŸ”´
	 NOINT	INT	
INT saves: 
['1 Male', '3 Female', '2 Old', '1 Working', '1 Medical', '1 Homeless', '5 Human']
NOINT saves: 
['1 Female', '1 Old', '1 Homeless', '2 Human']



How much data did we lose with all these joins? Shouldn't be any - all we're doing is dividing the dataset in half.

In [26]:
print("df: {}".format(df.shape))
print("df_endo: {}".format(df_endo.shape))
print("df_exo: {}".format(df_exo.shape))
print("df_joined: {}".format(df_joined.shape))

df: (1544920, 42)
df_endo: (772460, 42)
df_exo: (772460, 16)
df_joined: (772460, 57)


Finally, let's make it easier to interpret the target variable. For each response, we know whether the user chose to save one set of characters (\_int) by intervention, or save another set (\_noint) by not intervening. Let's call that variable "Intervened" to indicate whether or not the user intervened (swerved the AV).

In [27]:
df_joined['Intervened'] = (df_joined['Saved_int'] == 1).astype(int)
df_joined = df_joined.drop(axis='columns', labels=['Saved_{}'.format(s) for s in ['int', 'noint']]+["Intervene"])
scenario_fields.remove('Saved')
df_joined.columns

Index(['session_count', 'ScenarioOrder', 'PedPed', 'Barrier', 'CrossingSignal',
       'AttributeLevel', 'ScenarioTypeStrict', 'ScenarioType', 'DefaultChoice',
       'NonDefaultChoice', 'DefaultChoiceIsOmission', 'DiffNumberOFCharacters',
       'Template', 'UserCountry3', 'NumberOfCharacters_int',
       'DescriptionShown_int', 'LeftHand_int', 'Male_int', 'Female_int',
       'Young_int', 'Old_int', 'Infancy_int', 'Pregnancy_int', 'Fat_int',
       'Fit_int', 'Working_int', 'Medical_int', 'Homeless_int', 'Criminal_int',
       'Human_int', 'Non-human_int', 'Passenger_int', 'Law Abiding_int',
       'Law Violating_int', 'NumberOfCharacters_noint',
       'DescriptionShown_noint', 'LeftHand_noint', 'Male_noint',
       'Female_noint', 'Young_noint', 'Old_noint', 'Infancy_noint',
       'Pregnancy_noint', 'Fat_noint', 'Fit_noint', 'Working_noint',
       'Medical_noint', 'Homeless_noint', 'Criminal_noint', 'Human_noint',
       'Non-human_noint', 'Passenger_noint', 'Law Abiding_noint',


Now choose which features to use:

In [28]:
# include user countries and other metadata or not? decided not - experts writing LFs are trying to decide most moral response, not predict what an individual from a certain background would do
# but if it helps generalization, technically useful... try both?
target = ["Intervened"]
features = ["Template", "UserCountry3"] + ["{}_{}".format(f, s) for f in scenario_fields for s in ["int", "noint"]]
cat_features = [
    "Template", "UserCountry3"
]
num_features = [f for f in features if f not in cat_features]
df_joined[features].head()

Template  \
ResponseID        ExtendedSessionID             UserID           UUID            
C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010 0     Desktop   
CDnbR339euts3x3c3 1295349308_1000284909295010.0 1000284909295010 1     Desktop   
CMChvaNp5QZup4QBw 1295349308_1000284909295010.0 1000284909295010 2     Desktop   
CZ3BfpjubdHMdqq6z 1295349308_1000284909295010.0 1000284909295010 3     Desktop   
Kvr4yzcYB8QAxykMg 1295349308_1000284909295010.0 1000284909295010 8     Desktop   

                                                                      UserCountry3  \
ResponseID        ExtendedSessionID             UserID           UUID                
C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010 0             MEX   
CDnbR339euts3x3c3 1295349308_1000284909295010.0 1000284909295010 1             MEX   
CMChvaNp5QZup4QBw 1295349308_1000284909295010.0 1000284909295010 2             MEX   
CZ3BfpjubdHMdqq6z 1295349308_1000284909295010.0 1000284909295010 3             MEX   
Kvr4yzcYB8QAxykMg 1295349308_1000284909295010.0 1000284909295010 8             MEX   

                                                                       NumberOfCharacters_int  \
ResponseID        ExtendedSessionID             UserID           UUID                           
C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010 0                          3   
CDnbR339euts3x3c3 1295349308_1000284909295010.0 1000284909295010 1                          5   
CMChvaNp5QZup4QBw 1295349308_1000284909295010.0 1000284909295010 2                          5   
CZ3BfpjubdHMdqq6z 1295349308_1000284909295010.0 1000284909295010 3                          4   
Kvr4yzcYB8QAxykMg 1295349308_1000284909295010.0 1000284909295010 8                          2   

                                                                       NumberOfCharacters_noint  \
ResponseID        ExtendedSessionID             UserID           UUID                             
C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010 0                            3   
CDnbR339euts3x3c3 1295349308_1000284909295010.0 1000284909295010 1                            2   
CMChvaNp5QZup4QBw 1295349308_1000284909295010.0 1000284909295010 2                            5   
CZ3BfpjubdHMdqq6z 1295349308_1000284909295010.0 1000284909295010 3                            4   
Kvr4yzcYB8QAxykMg 1295349308_1000284909295010.0 1000284909295010 8                            2   

                                                                      DescriptionShown_int  \
ResponseID        ExtendedSessionID             UserID           UUID                        
C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010 0                       0   
CDnbR339euts3x3c3 1295349308_1000284909295010.0 1000284909295010 1                       0   
CMChvaNp5QZup4QBw 1295349308_1000284909295010.0 1000284909295010 2                       0   
CZ3BfpjubdHMdqq6z 1295349308_1000284909295010.0 1000284909295010 3                       0   
Kvr4yzcYB8QAxykMg 1295349308_1000284909295010.0 1000284909295010 8                       0   

                                                                      DescriptionShown_noint  \
ResponseID        ExtendedSessionID             UserID           UUID                          
C4yoxBdFMTE3tAjEu 1295349308_1000284909295010.0 1000284909295010 0                         0   
CDnbR339euts3x3c3 1295349308_1000284909295010.0 1000284909295010 1                         0   
CMChvaNp5QZup4QBw 1295349308_1000284909295010.0 1000284909295010 2                         0   
CZ3BfpjubdHMdqq6z 1295349308_1000284909295010.0 1000284909295010 3                         0   
Kvr4yzcYB8QAxykMg 1295349308_1000284909295010.0 1000284909295010 8                         0   

                                                                      LeftHand_int  \
ResponseID        ExtendedSessionID             UserID           UUID                
C4yoxBdFMTE3tA

Now explicitly type cast and deal with NA's or missing values:

In [29]:
# transform numerical data types
df_proc = df_joined.loc[:, features + target]

def transform_num(df, num_features):
    # convert to numeric, changing literals to NaN
    for f in num_features:
        df.loc[:, f] = pd.to_numeric(df.loc[:, f], errors='coerce')
    # are there any nan in the numerical features? usually just one
    nan = df[df[num_features].isna().any(axis=1)]
    print("Dropping these NaN:")
    display(nan)
    return df.dropna(axis=0, how='any', subset=num_features)

df_proc = transform_num(df_proc, num_features)
df_proc.columns

/opt/miniconda3/envs/hmm/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2886: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


Dropping these NaN:


Template  \
ResponseID        ExtendedSessionID      UserID     UUID               
9XqeeDRQxdjXGtysP -1132097515_1001529134 1001529134 182                
KQCdXCp4BcnqmmEky -1132097515_1001529134 1001529134 184                
Ljhao6hyR8PAtMEP4 -1132097515_1001529134 1001529134 185                
MB9o7ub7KkvdWaD6j -1132097515_1001529134 1001529134 186                
PRPQctv4Jzbxo7pMM -1132097515_1001529134 1001529134 187                
...                                                              ...   
o2i62HYZFCaiLSAeD 970197532_997534616    997534616  1541867            
pPGge5qxnWErpnWSs 970197532_997534616    997534616  1541868            
rccbnRp8ht7kyyXux 970197532_997534616    997534616  1541869            
wrDF6jHGJWtRftMf8 970197532_997534616    997534616  1541870            
zjyQQeGpKL6JxixCo 970197532_997534616    997534616  1541871            

                                                            UserCountry3  \
ResponseID        ExtendedSessionID      UserID     UUID                   
9XqeeDRQxdjXGtysP -1132097515_1001529134 1001529134 182              ROU   
KQCdXCp4BcnqmmEky -1132097515_1001529134 1001529134 184              ROU   
Ljhao6hyR8PAtMEP4 -1132097515_1001529134 1001529134 185              ROU   
MB9o7ub7KkvdWaD6j -1132097515_1001529134 1001529134 186              ROU   
PRPQctv4Jzbxo7pMM -1132097515_1001529134 1001529134 187              ROU   
...                                                                  ...   
o2i62HYZFCaiLSAeD 970197532_997534616    997534616  1541867          MEX   
pPGge5qxnWErpnWSs 970197532_997534616    997534616  1541868          MEX   
rccbnRp8ht7kyyXux 970197532_997534616    997534616  1541869          MEX   
wrDF6jHGJWtRftMf8 970197532_997534616    997534616  1541870          MEX   
zjyQQeGpKL6JxixCo 970197532_997534616    997534616  1541871          MEX   

                                                             NumberOfCharacters_int  \
ResponseID        ExtendedSessionID      UserID     UUID                              
9XqeeDRQxdjXGtysP -1132097515_1001529134 1001529134 182                           1   
KQCdXCp4BcnqmmEky -1132097515_1001529134 1001529134 184                           2   
Ljhao6hyR8PAtMEP4 -1132097515_1001529134 1001529134 185                           2   
MB9o7ub7KkvdWaD6j -1132097515_1001529134 1001529134 186                           3   
PRPQctv4Jzbxo7pMM -1132097515_1001529134 1001529134 187                           1   
...                                                                             ...   
o2i62HYZFCaiLSAeD 970197532_997534616    997534616  1541867                       1   
pPGge5qxnWErpnWSs 970197532_997534616    997534616  1541868                       5   
rccbnRp8ht7kyyXux 970197532_997534616    997534616  1541869                       2   
wrDF6jHGJWtRftMf8 970197532_997534616    997534616  1541870                       1   
zjyQQeGpKL6JxixCo 970197532_997534616    997534616  1541871                       1   

                                                             NumberOfCharacters_noint  \
ResponseID        ExtendedSessionID      UserID     UUID                                
9XqeeDRQxdjXGtysP -1132097515_1001529134 1001529134 182                             5   
KQCdXCp4BcnqmmEky -1132097515_1001529134 1001529134 184                             2   
Ljhao6hyR8PAtMEP4 -1132097515_1001529134 1001529134 185                             2   
MB9o7ub7KkvdWaD6j -1132097515_1001529134 1001529134 186                             3   
PRPQctv4Jzbxo7pMM -1132097515_1001529134 1001529134 187                             1   
...                                                                               ...   
o2i62HYZFCaiLSAeD 970197532_997534616    997534616  1541867                         1   
pPGge5qxnWErpnWSs 970197532_997534616    997534616  1541868                         3   
rccbnRp8ht7kyyXux 970197532_997534616    997534616  1541869                         2   
wrDF6jHGJWtRftMf8 97019

Index(['Template', 'UserCountry3', 'NumberOfCharacters_int',
       'NumberOfCharacters_noint', 'DescriptionShown_int',
       'DescriptionShown_noint', 'LeftHand_int', 'LeftHand_noint', 'Male_int',
       'Male_noint', 'Female_int', 'Female_noint', 'Young_int', 'Young_noint',
       'Old_int', 'Old_noint', 'Infancy_int', 'Infancy_noint', 'Pregnancy_int',
       'Pregnancy_noint', 'Fat_int', 'Fat_noint', 'Fit_int', 'Fit_noint',
       'Working_int', 'Working_noint', 'Medical_int', 'Medical_noint',
       'Homeless_int', 'Homeless_noint', 'Criminal_int', 'Criminal_noint',
       'Human_int', 'Human_noint', 'Non-human_int', 'Non-human_noint',
       'Passenger_int', 'Passenger_noint', 'Law Abiding_int',
       'Law Abiding_noint', 'Law Violating_int', 'Law Violating_noint',
       'Intervened'],
      dtype='object')

### Train/Test Split

A standard train test split for testing:

In [30]:
import hmm.classification
importlib.reload(hmm.classification)
from hmm.classification import train_test_val_dev_split

def make_X_y(df):
    X = df.drop(labels=["Intervened"], axis='columns', inplace=False)
    y = df["Intervened"]
    return X, y

X, y = make_X_y(df_proc)
X_train, X_test, X_val, X_dev, y_train, y_test, y_val, y_dev = train_test_val_dev_split(X, y)
display(X_train.head())
display(y_train.head())
X.columns

Template  \
ResponseID        ExtendedSessionID              UserID           UUID               
zMxxfgF5toKascmBC -1560484671_5278886028275740.0 5278886028275740 990986    Mobile   
JKxFztgS4cZc6iGx6 694893935_9451969873639508.0   9451969873639510 1482082  Desktop   
ZnvDbZjQS24j6rXtg -688785863_4707035014871508.0  4707035014871510 920524    Mobile   
nbFBW7od8dNXobhCE 1228232883_2636069680125507.0  2636069680125510 434636   Desktop   
w9F5iLSRTWPPRwxTK -2023517918_2912710996256167.0 2912710996256170 511256    Mobile   

                                                                          UserCountry3  \
ResponseID        ExtendedSessionID              UserID           UUID                   
zMxxfgF5toKascmBC -1560484671_5278886028275740.0 5278886028275740 990986           DZA   
JKxFztgS4cZc6iGx6 694893935_9451969873639508.0   9451969873639510 1482082          NOR   
ZnvDbZjQS24j6rXtg -688785863_4707035014871508.0  4707035014871510 920524           JPN   
nbFBW7od8dNXobhCE 1228232883_2636069680125507.0  2636069680125510 434636           AUS   
w9F5iLSRTWPPRwxTK -2023517918_2912710996256167.0 2912710996256170 511256           BRA   

                                                                           NumberOfCharacters_int  \
ResponseID        ExtendedSessionID              UserID           UUID                              
zMxxfgF5toKascmBC -1560484671_5278886028275740.0 5278886028275740 990986                        2   
JKxFztgS4cZc6iGx6 694893935_9451969873639508.0   9451969873639510 1482082                       5   
ZnvDbZjQS24j6rXtg -688785863_4707035014871508.0  4707035014871510 920524                        1   
nbFBW7od8dNXobhCE 1228232883_2636069680125507.0  2636069680125510 434636                        5   
w9F5iLSRTWPPRwxTK -2023517918_2912710996256167.0 2912710996256170 511256                        2   

                                                                           NumberOfCharacters_noint  \
ResponseID        ExtendedSessionID              UserID           UUID                                
zMxxfgF5toKascmBC -1560484671_5278886028275740.0 5278886028275740 990986                          2   
JKxFztgS4cZc6iGx6 694893935_9451969873639508.0   9451969873639510 1482082                         5   
ZnvDbZjQS24j6rXtg -688785863_4707035014871508.0  4707035014871510 920524                          1   
nbFBW7od8dNXobhCE 1228232883_2636069680125507.0  2636069680125510 434636                          4   
w9F5iLSRTWPPRwxTK -2023517918_2912710996256167.0 2912710996256170 511256                          2   

                                                                           DescriptionShown_int  \
ResponseID        ExtendedSessionID              UserID           UUID                            
zMxxfgF5toKascmBC -1560484671_5278886028275740.0 5278886028275740 990986                    0.0   
JKxFztgS4cZc6iGx6 694893935_9451969873639508.0   9451969873639510 1482082                   0.0   
ZnvDbZjQS24j6rXtg -688785863_4707035014871508.0  4707035014871510 920524                    1.0   
nbFBW7od8dNXobhCE 1228232883_2636069680125507.0  2636069680125510 434636                    0.0   
w9F5iLSRTWPPRwxTK -2023517918_2912710996256167.0 2912710996256170 511256                    0.0   

                                                                           DescriptionShown_noint  \
ResponseID        ExtendedSessionID              UserID           UUID                              
zMxxfgF5toKascmBC -1560484671_5278886028275740.0 5278886028275740 990986                      0.0   
JKxFztgS4cZc6iGx6 694893935_9451969873639508.0   9451969873639510 1482082                     0.0   
ZnvDbZjQS24j6rXtg -688785863_4707035014871508.0  4707035014871510 920524                      1.0   
nbFBW7od8dNXobhCE 1228232883_2636069680125507.0  2636069680125510 434636                      0.0   
w9F5iLSRTWPPRwxTK -2023517918_2912710996256167.0 2912710996256170 511256                      0.0   

        

ResponseID         ExtendedSessionID               UserID            UUID   
zMxxfgF5toKascmBC  -1560484671_5278886028275740.0  5278886028275740  990986     0
JKxFztgS4cZc6iGx6  694893935_9451969873639508.0    9451969873639510  1482082    1
ZnvDbZjQS24j6rXtg  -688785863_4707035014871508.0   4707035014871510  920524     1
nbFBW7od8dNXobhCE  1228232883_2636069680125507.0   2636069680125510  434636     1
w9F5iLSRTWPPRwxTK  -2023517918_2912710996256167.0  2912710996256170  511256     0
Name: Intervened, dtype: int64

Index(['Template', 'UserCountry3', 'NumberOfCharacters_int',
       'NumberOfCharacters_noint', 'DescriptionShown_int',
       'DescriptionShown_noint', 'LeftHand_int', 'LeftHand_noint', 'Male_int',
       'Male_noint', 'Female_int', 'Female_noint', 'Young_int', 'Young_noint',
       'Old_int', 'Old_noint', 'Infancy_int', 'Infancy_noint', 'Pregnancy_int',
       'Pregnancy_noint', 'Fat_int', 'Fat_noint', 'Fit_int', 'Fit_noint',
       'Working_int', 'Working_noint', 'Medical_int', 'Medical_noint',
       'Homeless_int', 'Homeless_noint', 'Criminal_int', 'Criminal_noint',
       'Human_int', 'Human_noint', 'Non-human_int', 'Non-human_noint',
       'Passenger_int', 'Passenger_noint', 'Law Abiding_int',
       'Law Abiding_noint', 'Law Violating_int', 'Law Violating_noint'],
      dtype='object')

### Labeling Model

Some sample labeling functions, constructed with the help of the effect sizes in the Moral Machine experiment.

In [31]:
import hmm.labeling.moralmachine as mm
import hmm.labeling.models
import hmm.labeling.utils
importlib.reload(hmm.labeling.moralmachine)
importlib.reload(hmm.labeling.models)
importlib.reload(hmm.labeling.utils)
import hmm.labeling.moralmachine as mm
import hmm.labeling.models
import hmm.labeling.utils

from hmm.labeling.models import Labeler
from snorkel.labeling import LFAnalysis

lfs = [
    mm.doctors,
    mm.utilitarian,
    mm.utilitarian_anthro,
    mm.inaction,
    mm.pedestrians,
    mm.females,
    mm.fitness,
    mm.status,
    mm.legal,
    mm.illegal,
    mm.youth,
    mm.criminals,
    mm.homeless,
    mm.pets,
    mm.spare_strollers,
    mm.spare_pregnant
]

labeler = Labeler(lfs)
L_train, L_dev, L_val = labeler.label([X_train, X_dev, X_val])
LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=y_dev.values).sort_values("Correct", ascending=False)

/opt/miniconda3/envs/hmm/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ| 106105/106105 [01:09<00:00, 1522.36it/s]


j Polarity  Coverage  Overlaps  Conflicts  Correct  \
inaction             3      [0]  1.000000  0.993554   0.853282    12145   
legal                8   [0, 1]  0.784069  0.784069   0.739888    11617   
females              5   [0, 1]  0.519848  0.519848   0.450635     9353   
pedestrians          4   [0, 1]  0.542617  0.542617   0.542617     7695   
utilitarian_anthro   2   [0, 1]  0.366231  0.366231   0.325819     7469   
illegal              9   [0, 1]  0.542617  0.542617   0.542617     7285   
youth               10   [0, 1]  0.280846  0.280846   0.245523     5722   
fitness              6   [0, 1]  0.323557  0.323557   0.287631     4606   
utilitarian          1   [0, 1]  0.213971  0.213971   0.193463     4243   
pets                13   [0, 1]  0.164059  0.164059   0.143213     3452   
status               7   [0, 1]  0.178497  0.178497   0.157952     3240   
doctors              0   [0, 1]  0.126965  0.126965   0.111471     2472   
homeless            12   [0, 1]  0.085159  0.085159   0.071851     1416   
spare_pregnant      15   [0, 1]  0.067478  0.067478   0.058393     1312   
spare_strollers     14   [0, 1]  0.059675  0.059675   0.053568     1215   
criminals           11   [0, 1]  0.005202  0.005202   0.004825       98   

                    Incorrect  Emp. Acc.  
inaction                14382   0.457835  
legal                    9182   0.558536  
females                  4437   0.678245  
pedestrians              6699   0.534598  
utilitarian_anthro       2246   0.768811  
illegal                  7109   0.506114  
youth                    1728   0.768054  
fitness                  3977   0.536642  
utilitarian              1433   0.747533  
pets                      900   0.793199  
status                   1495   0.684266  
doctors                   896   0.733967  
homeless                  843   0.626826  
spare_pregnant            478   0.732961  
spare_strollers           368   0.767530  
criminals                  40   0.710145

In [32]:
# using the validation set (since tuning is done)
analysis = LFAnalysis(L=L_val, lfs=lfs).lf_summary(Y=y_val.values)
analysis.to_csv("../figures/data/lfanalysis.csv")
# labeling density
pd.DataFrame(L_dev, columns=[lf.name for lf in lfs]).to_csv("../figures/data/mm-density.csv")

## Aggregation

Recall that there are no true labels for this problem - really, we're just measuring similarity of the heuristic labels to real voter's responses. 

**Baseline**: majority label voting.

In [33]:
from snorkel.labeling import MajorityLabelVoter

model_majority = MajorityLabelVoter()
preds_train = model_majority.predict(L=L_train)

**Label Model**: Snorkel aggregator. Chooses weights to combine the labeling functions based on learned conditional probabilities.

In [34]:
# cardinality is num classes
importlib.reload(hmm.labeling.models)

model_label = labeler.fit(L_train, Y_dev=y_dev.values, fit_params={'n_epochs': 200, 'log_freq': 50})
analysis = LFAnalysis(L=L_val, lfs=lfs).lf_summary(Y=y_val.values)
analysis['weight'] = pd.Series(model_label.get_weights(), index=[lf.name for lf in lfs])
analysis.to_csv('../figures/data/mm-weights.csv')
analysis.sort_values('Emp. Acc.')
# X[['Passenger_int', 'Passenger_noint', 'Law Abiding_int', 'Law Abiding_noint']]

j Polarity  Coverage  Overlaps  Conflicts  Correct  \
inaction             3      [0]  1.000000  0.993704   0.853692    48665   
illegal              9   [0, 1]  0.549154  0.549154   0.549154    28947   
pedestrians          4   [0, 1]  0.549154  0.549154   0.549154    30802   
fitness              6   [0, 1]  0.323444  0.323444   0.288064    18543   
legal                8   [0, 1]  0.786626  0.786626   0.743330    46718   
homeless            12   [0, 1]  0.090099  0.090099   0.076311     6025   
status               7   [0, 1]  0.178201  0.178201   0.156826    12741   
females              5   [0, 1]  0.520918  0.520918   0.452523    37583   
doctors              0   [0, 1]  0.125866  0.125866   0.110975     9717   
criminals           11   [0, 1]  0.005457  0.005457   0.005212      425   
spare_pregnant      15   [0, 1]  0.068385  0.068385   0.060384     5404   
utilitarian          1   [0, 1]  0.216531  0.216531   0.195061    17302   
spare_strollers     14   [0, 1]  0.058141  0.058141   0.052514     4745   
utilitarian_anthro   2   [0, 1]  0.366411  0.366411   0.326667    29944   
youth               10   [0, 1]  0.276811  0.276811   0.241176    22759   
pets                13   [0, 1]  0.161793  0.161793   0.142057    13620   

                    Incorrect  Emp. Acc.    weight  
inaction                57440   0.458649  0.457835  
illegal                 29321   0.496791  0.684168  
pedestrians             27466   0.528626  0.649047  
fitness                 15776   0.540313  0.589125  
legal                   36747   0.559732  0.532390  
homeless                 3535   0.630230  0.661318  
status                   6167   0.673842  0.834306  
females                 17689   0.679965  0.754420  
doctors                  3638   0.727593  0.923439  
criminals                 154   0.734024  0.850435  
spare_pregnant           1852   0.744763  0.965728  
utilitarian              5673   0.753079  0.928830  
spare_strollers          1424   0.769168  1.000000  
utilitarian_anthro       8934   0.770204  0.894260  
youth                    6612   0.774880  0.764100  
pets                     3547   0.793383  0.970447

How much does the label model improve on the majority voter?

In [35]:
importlib.reload(hmm.labeling.models)
from hmm.labeling.models import Labeler

for model in [model_majority, model_label]:
    Labeler.score(model, L_val, y_val)

MajorityLabelVoter() Vote Accuracy:            63.5
LabelModel() Vote Accuracy:            64.1


### Eye Test - Debugging Label Model

A handy function for viewing pairwise alternatives:

Now, use the label model to create probabilistic labels for the dev set. Rounding off, create binary predictions.

In [36]:
from snorkel.analysis import get_label_buckets

threshold = 0.5
probs_dev = model_label.predict_proba(L=L_dev)
preds_dev = probs_dev >= threshold

Create label buckets for eyeball debugging (groups TP, FP, TN, FN).

In [69]:
# confusion matrix
print(labeler.get_confusion_matrix(L_dev, y_dev))
buckets = labeler.get_label_buckets(L_dev, y_dev)

Predicted  False  True 
Actual                 
False       9881   2264
True        8136   6246


What is the accuracy for each scenario type?

In [70]:
# false negatives
df_fn_dev = X_dev.iloc[buckets[(1, 0)]]
# false positives
df_fp_dev = X_dev.iloc[buckets[(0, 1)]]
df_n_dev = X_dev.iloc[np.concatenate([buckets[(1, 0)], buckets[(0, 1)]])]
# acc = 1 - Neg / Total for each scenario type
acc = 1 - df_joined.loc[df_n_dev.index, 'ScenarioType'].value_counts() / df_joined.loc[X_dev.index, 'ScenarioType'].value_counts()
acc.sort_values()

Fitness          0.448202
Random           0.562704
Gender           0.568033
Social Status    0.587887
Age              0.658598
Species          0.699736
Utilitarian      0.702272
Name: ScenarioType, dtype: float64

#### False Negatives
Here, the user chose to intervene, while the label model did not. 

In [71]:
# get the corresponding posteriori probability for each false negative
df_fn_dev.loc[:,"probability"] = probs_dev[buckets[(1, 0)], 1]
# check out a few
pictofy(df_fn_dev.sample())

What should the self-driving car do? (ScenarioType: Unknown)

	    ðŸš˜ 
	   |	\ 
	   v 	 v
	ðŸ”´ðŸš¸ 	ðŸš§ðŸ”´
	 NOINT	INT	
INT saves: 
['3 Male', '2 Female', '4 Fit', '5 Human']
NOINT saves: 
['3 Male', '2 Female', '4 Fat', '5 Human']



/opt/miniconda3/envs/hmm/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2858: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
/opt/miniconda3/envs/hmm/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/miniconda3/envs/hmm/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel

Which scenario types does the model tend to get wrong?

In [72]:
df_joined.loc[df_fn_dev.index, 'ScenarioType'].value_counts().plot.pie()

#### False Positives
Here, the user chose not to intervene, but the label model did.

In [73]:
# get the corresponding posteriori probability for each false positive
df_fp_dev.loc[:,"probability"] = probs_dev[buckets[(0, 1)], 1]
# check out a few
pictofy(df_fp_dev.sample(random_state=3))

What should the self-driving car do? (ScenarioType: Unknown)

	    ðŸš˜ 
	   |	\ 
	   v 	 v
	ðŸ”´ðŸš§ 	ðŸš¸ðŸŸ¢
	 NOINT	INT	
INT saves: 
['3 Male', '1 Female', '1 Old', '4 Human']
NOINT saves: 
['3 Male', '1 Female', '3 Young', '4 Human']



In [74]:
df_joined.loc[df_fp_dev.index, 'ScenarioType'].value_counts().plot.pie()

## Classification

https://www.snorkel.org/use-cases/01-spam-tutorial#5-training-a-classifier

### Featurization

Let's design a simple machine learning classifier for this problem, then test it on both the gold standard labels and the heuristic labels.

In [75]:
import hmm.classification
importlib.reload(hmm.classification)
from hmm.classification import Classifier

clf = Classifier(features, num_features, cat_features)

#### Training on Gold Labels

Using just the labels (no label model):

In [76]:
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

KeyboardInterrupt: 

#### Training on Heuristic Labels

Using the label model, filter out unlabeled points:

In [ ]:
X_train_filtered, probs_train_filtered = labeler.filter_probs(X_train, L_train)
# check if filtered X_train has same columns
# X_train_filtered.columns == X_train.columns
# how many were filtered?
X_train.shape[0] - X_train_filtered.shape[0]

In [ ]:
preds_train_filtered = Labeler.probs_to_preds(probs_train_filtered)
clf.fit(X_train_filtered, preds_train_filtered)
clf.score(X_test, y_test)

### Grid Search Testing

**Which labeling functions add the most predictive performance?**

In [44]:
acc_full = Labeler.score(model_label, L_val, y_val)
for lf in lfs:
    lfs_perturb = [l for l in lfs if l != lf]
    l = Labeler(lfs_perturb)
    L_train, L_dev, L_val = l.label([X_train, X_dev, X_val], verbose=False)
    lm = labeler.fit(L_train)
    acc_perturb = Labeler.score(lm, L_val, y_val, verbose=False)
    print("{}: {}".format(lf.name, acc_full - acc_perturb))

LabelModel() Vote Accuracy:            59.0
doctors: 0.00027973592928276503


KeyboardInterrupt: 

**Which models perform best?**

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

for name, model in {
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Log Reg": LogisticRegression(C=.001),
    "KNN": KNeighborsClassifier(3),
    "SVC Linear": SVC(kernel="linear", C=0.025),
    "SVC Nonlinear": SVC(gamma=2, C=1),
    "GP": GaussianProcessClassifier(1.0 * RBF(1.0)),
    "Decision Tree": DecisionTreeClassifier(max_depth=5),
    "RF": RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    "MLP": MLPClassifier(alpha=1, max_iter=1000),
    "AdaBoost": AdaBoostClassifier(),
    "Naive Bayes": GaussianNB(),
    "QDA": QuadraticDiscriminantAnalysis()
}.items():
    print("## {} ##".format(name))
    clf = Classifier(features, num_features, cat_features, clf=model)
    
    clf.fit(X_train, y_train)
    acc = clf.score(X_test, y_test, verbose=False)
    print("Accuracy with gold labels: {}".format(acc))
    
    clf.fit(X_train_filtered, preds_train_filtered)
    acc = clf.score(X_test, y_test, verbose=False)
    print("Accuracy with heuristic labels: {}".format(acc))

## Random Forest ##
Accuracy with gold labels: 0.6969270720623242
Accuracy with heuristic labels: 0.6713373728630166
## Log Reg ##


/Users/steed/miniconda3/envs/hmm/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy with gold labels: 0.7052045011902186


/Users/steed/miniconda3/envs/hmm/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy with heuristic labels: 0.6623566327634711
## KNN ##


**How does performance change as the number of voters is increased?** 

In this case, evaluate performance voter-wise by splitting the data after stratifying by voter.

In [154]:
data = []
users = df_proc.groupby(['UserID'])
a = np.arange(users.ngroups)
np.random.shuffle(a)
for i in range(2,8):
    n = 2**i
    print("## Testing N={}##".format(n))
    n_respondents = df_proc[users.ngroup().isin(a[:n])]
    # checking to make sure we're getting complete, single sessions
    # some users may have more than one session...
#     n_respondents.groupby('UserID').count().describe()
    X, y = make_X_y(n_respondents)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.4, stratify=X.index.get_level_values('ExtendedSessionID')
    )
#     y_train.groupby('ExtendedSessionID').count()
    L_train, L_test = labeler.label([X_train, X_test])
    lm = labeler.fit(L_train)
    Labeler.score(lm, L_test, y_test, verbose=False)
    X_train_filtered, probs_train_filtered = labeler.filter_probs(X_train, L_train)
    # baseline gold model accuracy
    print("Gold accuracy")
    clf.fit(X_train, y_train)
    acc_gold = clf.score(X_test, y_test)
    # label model accuracy
    print("LF accuracy")
    preds_train_filtered = Labeler.probs_to_preds(probs_train_filtered)
    clf.fit(X_train_filtered, preds_train_filtered)
    acc_lf = clf.score(X_test, y_test)
    data.append((n, acc_gold, acc_lf))
data

/Users/steed/miniconda3/envs/hmm/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ| 26/26 [00:00<00:00, 861.93it/s]

## Testing N=4##


Gold accuracy
Test Accuracy: 61.5%
LF accuracy


/Users/steed/miniconda3/envs/hmm/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ| 47/47 [00:00<00:00, 903.56it/s]

Test Accuracy: 53.8%
## Testing N=8##


...

/Users/steed/miniconda3/envs/hmm/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
  0%|          | 0/1021 [00:00<?, ?it/s]

Test Accuracy: 57.9%
LF accuracy
Test Accuracy: 62.8%
## Testing N=128##


100%|â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ| 682/682 [00:00<00:00, 848.21it/s]


Gold accuracy
Test Accuracy: 62.8%
LF accuracy
Test Accuracy: 63.3%


[(4, 0.6153846153846154, 0.5384615384615384),
 (8, 0.6595744680851063, 0.5531914893617021),
 (16, 0.6292134831460674, 0.6741573033707865),
 (32, 0.6569767441860465, 0.6511627906976745),
 (64, 0.5787965616045845, 0.6275071633237822),
 (128, 0.6275659824046921, 0.6334310850439883)]

## Kim et al. Implementation

A replication of the model used by [Kim et al.](https://www.mendeley.com/viewer/?fileId=1b0d79d6-97a3-f7e5-3008-2e14db1a767b&documentId=3f4000f0-5830-358b-a8ea-b031e3ab5403) for estimating voter preferences. Will compare accuracy to Snorkel method.

Here, modeling _utilities_ as $u(\Theta_i) = w^\intercal F(\Theta_i)$, where $\Theta_y$ is the character vector for instance $y$ and $F : \Theta \to \Lambda$ is the mapping from the character vector to an abstract vector. This is accomplished by $F(\Theta) = A\Theta$, where $A$ is a binary matrix denoting the presence of abstract characterisitics.

### Benchmark 1

$u(\Theta) = w^{c\intercal}\Theta$, where $w^c \sim N_k(\mu, \sigma^2I)$.

In [67]:
## TODO: how to estimate w^c?